<a href="https://colab.research.google.com/github/SeohuiPark/MLDLstudy/blob/main/AGnews_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


전처리 과정 참고 link https://m.blog.naver.com/gkvmsp/221485860027

전처리 참고 2 : https://jdjin3000.tistory.com/15


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:150% !important; }</style>"))

In [ ]:
# [ base module ]
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_pandas
from tqdm.notebook import tqdm
import pickle

# [ sklearn ]
from sklearn.model_selection import train_test_split

# [ torch ]
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

from torchtext.legacy import data, datasets
from torchtext.legacy.data import Iterator, Field, TabularDataset
from torchsummary import summary as summary_

# [ NLP ]
import re
import nltk
import gensim

nltk.download('averaged_perceptron_tagger') # pos tagging
nltk.download('wordnet') # lemmatization
nltk.download('stopwords') # stopwords

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def data_load(data_path):

    ### data load 
    data_path = "/content/drive/MyDrive/Colab Notebooks/2022_pytorch/data/"
    data = pd.read_csv(data_path + 'AGnews_train.csv', header = None)
    data.columns = ['label','title','description']
    display(data.head(10))

    # topics = {1: 'world',
    #           2: 'sports',
    #           3: 'business',
    #           4: 'science'} # Originally "Sci/Tech"

    print(data.shape)
    print(data['label'].value_counts())
    data['all_text'] = data['title'] + ' ' + data['description'] ### 제목과 기사 내용 합치기     

    return data 

In [ ]:
global final_word_list
final_word_list = []

def lemmatization(POS_list, DROP_1WORD):
    lemma_list = []
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    for sentence in POS_list:
        #Lemmatize 함수를 사용할 때, 해당 토큰이 어떤 품사인지 알려줄 수 있다. 만약 품사를 알려주지 않는다면 올바르지 않은 결과가 반환될 수 있다.
        #Lemmatize 함수가 입력받는 품사는 동사, 형용사, 명사, 부사 뿐이다. ===>  각각 v, a, n, r로 입력받는다.
        #nltk.pos_tag로 반환받는 품사 중, 형용사는 J로 시작하기 때문에 lemmatize 전에 a로 바꿔줘야 한다.

        func_j2a = lambda x : x if x != 'j' else 'a'
        pos_contraction = [(token, func_j2a(POS.lower()[0])) for token, POS in sentence if POS[0] in ['V', 'J', 'N', 'R']]
        
        if DROP_1WORD: ### 글자수 1인 경우에는 drop 
            for token, POS in pos_contraction :
                if len(token) != 1: 
                    lemma_list.append(lemmatizer.lemmatize(token, POS))
        else: 
            lemma_list.append([lemmatizer.lemmatize(token, POS) for token, POS in pos_contraction])
    return lemma_list


def remove_stopwords(lemma_list):
    no_stopword_list = list()
    stop_words = set(nltk.corpus.stopwords.words('english')) # 불용어 불러오기 
    for lemma_word in lemma_list:
        if lemma_word not in stop_words:
            no_stopword_list.append(lemma_word)
    return no_stopword_list

def text_prep(text): # input shape - sentence 

    DROP_1WORD = True
    # text preprocessing 
    text = text.lower()
    text = text.replace('<br />', ' ')
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\n', ' ', text)

    # pos checking - lemmatization
    pos_text = nltk.pos_tag(text.split())
    pos_text_list = [pos_text]
    lem_text_list = lemmatization(pos_text_list, DROP_1WORD)
    no_stopword_list = remove_stopwords(lem_text_list)

    ## save word list 
    final_word_list.append(no_stopword_list)
    # print(no_stopword_list)

    return ' '.join(no_stopword_list)

In [ ]:
if __name__ == "__main__":
    data_path = "/content/drive/MyDrive/Colab Notebooks/2022_pytorch/data/"
    data = data_load(data_path)

    ### preprocessing
    tqdm.pandas()
    word_series = data['all_text'].progress_apply(text_prep) 
    data['prep_text'] = word_series

    ### save prep data
    final_df = data[['prep_text','label']]
    display(final_df)
    final_df.to_csv(data_path+'AGnews_prep4.csv', index = False)

,label,title,description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
5,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...
6,3,Money Funds Fell in Latest Week (AP),AP - Assets of the nation's retail money marke...
7,3,Fed minutes show dissent over inflation (USATO...,USATODAY.com - Retail sales bounced back a bit...
8,3,Safety Net (Forbes.com),Forbes.com - After earning a PH.D. in Sociolog...
9,3,Wall St. Bears Claw Back Into the Black,"NEW YORK (Reuters) - Short-sellers, Wall Stre..."


(120000, 3)
3    30000
4    30000
2    30000
1    30000
Name: label, dtype: int64


  0%|          | 0/120000 [00:00<?, ?it/s]

,prep_text,label
0,wall st bear claw back black reuters reuters s...,3
1,carlyle look commercial aerospace reuters reut...,3
2,oil economy cloud stock outlook reuters reuter...,3
3,iraq halt oil export main southern pipeline re...,3
4,oil price soar time record pose new menace eco...,3
...,...,...
119995,pakistan musharraf say win quit army chief kar...,1
119996,renteria sign top shelf deal red sox general m...,2
119997,saban go dolphins yet miami dolphin put courts...,2
119998,today nfl game pittsburgh ny giant time line s...,2
